In [1]:
import pandas as pd

my_df = pd.read_csv('spb_dp_corpus.csv')
print(my_df[-7:])
print(my_df[:7])

                                                   link  \
1478  https://www.dp.ru/a/2020/10/27/Obvinjaemogo_v_...   
1479  https://www.dp.ru/a/2020/11/08/Peterburgskie_s...   
1480  https://www.dp.ru/a/2020/11/11/Verojatnim_kand...   
1481  https://www.dp.ru/a/2020/11/12/V_Peterburge_ot...   
1482  https://www.dp.ru/a/2020/11/20/Na_severe_Peter...   
1483  https://www.dp.ru/a/2020/11/13/vistavka_JUBILE...   
1484  https://www.dp.ru/a/2020/11/20/Bolee_43_tis__p...   

                                               headline  \
1478  Обвиняемого в убийстве петербургского преподав...   
1479  Петербургские школы начали принимать заявления...   
1480  Вероятным кандидатом в вице-губернаторы назвал...   
1481  В Петербурге от коронавируса скончалось ещё 68...   
1482  На севере Петербурга уходивший от погони водит...   
1483  Афиша на 13 – 20 ноября. Главные события выход...   
1484  Более 43 тыс. петербуржцев сдали анализы на ко...   

                                                articl

In [2]:
# посмотреть на тэги 
all_tags = set(my_df['tag'].tolist())
print(all_tags) # все тэги у статей
print(my_df['tag'].value_counts())  # распределение тэгов

{'Авторынок Петербурга', 'Агропром', 'Полиция', 'Детские сады, ясли', 'Коронавирус', 'Мосты', 'Инфраструктура', 'Ресторанный бизнес', 'Коворкинги', 'Бюджет Петербурга', 'Рынок труда', 'Водный транспорт', 'Ветер', 'Уборка дорог и улиц', 'Суд', 'Стройка - Итоги', 'Удаленная работа', 'Образование', 'Памятник', 'Демография', 'Транспорт', 'Взрыв', 'Наука', 'Парки', 'Девелопмент', 'Уставный суд', 'Евро 2020', 'Аутсорсинг', 'Убийство', 'дело историка Соколова', 'Инвестиции', 'Таможня', 'Протесты в Белоруссии', 'Наркотики', 'Скандалы', 'Гостиничный бизнес', 'Алкоголь', 'Новый год', 'Аресты', 'Взятка', 'Юридические услуги', 'Гаджеты', 'Арбитраж', 'Связь', 'Ретейл', 'Ипотека', 'Пулково', 'Где жить', 'PR-Елка', 'Афиша', 'Здоровье', 'Ремонт дорог', 'Технологии', 'Аварии', 'Фармацевтика', 'Газ', 'Жилая недвижимость', 'Жилье', 'Штрафы', 'Долгострой', 'Новости СПб', 'Аукционы и торги', 'Стартапы', 'Перестановки в правительстве', 'Экономика', 'Общепит', 'Реставрация', 'Законодательное собрание', 'Поли

## Natasha

In [3]:
from natasha import MorphVocab,  NamesExtractor, NewsNERTagger, NewsEmbedding, Doc, Segmenter, NewsMorphTagger

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()
morph_vocab = MorphVocab() 
morph_tagger = NewsMorphTagger(emb)

In [4]:
from collections import defaultdict, Counter


def analysing_tags(tag):
    """
    на входе строка с названием тэга
    на выходе df с информацией о ner
    """
    # все статьи этой категории превращаем в строке
    tag_df = my_df[my_df['tag'] == tag]
    tag2str = tag_df.to_string(columns=['article'], index=False)
    
    # извлекаем ner 
    doc = Doc(tag2str)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    doc.tag_morph(morph_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    spans = [(x.normal, x.type) for x in doc.spans]
    
    # для каждого типа именнованных сущностей считает число предстаителей в тексте 
    all_ners = defaultdict(Counter)
    for span in spans:
        all_ners[span[1]].update([span[0]])
        
    # делаем так, чтобы можно было красиво записать и удобно посмотреть 
    prep_res = []
    for ner_type in all_ners:
        for name in all_ners[ner_type]:
            x = {'type': ner_type, 'name': name, 'count':all_ners[ner_type][name]}
            prep_res.append(x)
    nice_res = pd.DataFrame(prep_res)
    
    return nice_res

## Анализ рубрик(тэгов)

### Коронавирус

In [5]:
ner_corona = analysing_tags('Коронавирус')

In [6]:
ner_corona.sort_values(by='count', ascending=False) # смотрим на самые популярные сущности для данного тэга

,type,name,count
0,LOC,Петербург,370
3,LOC,Северная столица,62
91,ORG,ДП,53
89,ORG,Смольного,45
4,LOC,Россия,42
...,...,...,...
166,ORG,COVID-запретов,1
165,ORG,НИИ Скорая помощь,1
164,ORG,НИИ,1
163,ORG,Политех,1


In [7]:
per = ner_corona[ner_corona['type']=='PER']
print(per.sort_values(by='count', ascending=False)[:20]) # смотрим на 20 самых популярных людей

    type              name  count
287  PER  Александр Беглов     33
280  PER      Олег Эргашев     24
288  PER            Беглов     18
330  PER   Дмитрий Лисовец     12
290  PER           Эргашев     11
331  PER           Лисовец      8
315  PER     Андрей Сарана      7
366  PER       Анна Попова      6
293  PER    Виталий Середа      6
316  PER            Сарана      6
281  PER      Евгений Елин      6
347  PER      Сергей Бабин      6
284  PER  Игорь Каграманян      5
351  PER    Ирина Потехина      5
285  PER    Владимир Путин      4
282  PER              Елин      4
339  PER          Мясников      3
292  PER  Дмитрий Зеленуха      3
375  PER   Михаил Охлопков      3
297  PER             Басты      3


### Происшествия

In [8]:
events = analysing_tags('Происшествия')

In [9]:
events.sort_values(by='count', ascending=False) 

,type,name,count
112,LOC,Петербург,106
113,LOC,Россия,48
5,ORG,МВД,47
4,ORG,ГУ,39
110,LOC,Ленобласть,36
...,...,...,...
128,LOC,Морис Торез,1
130,LOC,Тореза,1
131,LOC,Буяницы,1
132,LOC,Волосовский район,1


In [10]:
org = ner_corona[ner_corona['type']=='ORG']
print(org.sort_values(by='count', ascending=False)[:20])

    type                      name  count
91   ORG                        ДП     53
89   ORG                 Смольного     45
95   ORG           Роспотребнадзор     25
98   ORG                  COVID-19     24
100  ORG                  Минздрав     24
87   ORG                  Смольном     19
84   ORG                  Ленэкспо     15
93   ORG                 ВКонтакте     13
92   ORG       Мариинская больница     12
142  ORG           Санкт-Петербург     10
126  ORG               РИА Новости      7
116  ORG                  Фонтанка      7
108  ORG  Законодательное собрание      7
125  ORG                  Смольный      6
175  ORG                 Интерфакс      5
86   ORG                        ВК      5
138  ORG                       СМИ      5
233  ORG                Первый мед      4
99   ORG   НИИ фтизиопульмонологии      4
122  ORG                      ТАСС      4


### Общество

In [11]:
society = analysing_tags('Общество')

In [12]:
society.sort_values(by='count', ascending=False) 

,type,name,count
1,LOC,Петербург,203
188,ORG,ДП,36
6,LOC,Северная столица,36
28,LOC,Россия,35
461,PER,Александр Беглов,25
...,...,...,...
267,ORG,ВКС,1
266,ORG,ИГИЛ,1
265,ORG,Рейс 9268,1
264,ORG,Когалымавиа,1


In [13]:
loc = ner_corona[ner_corona['type']=='LOC']
print(loc.sort_values(by='count', ascending=False)[:20])

   type                     name  count
0   LOC                Петербург    370
3   LOC         Северная столица     62
4   LOC                   Россия     42
5   LOC                   Москва     24
9   LOC               Ленобласть     24
8   LOC                       РФ     23
7   LOC                     Нева     15
75  LOC                  Карелия     13
6   LOC              Подмосковье      9
16  LOC  Калининградская область      3
51  LOC                   Кремль      3
55  LOC                      США      3
56  LOC          Санкт-Петербург      3
19  LOC                    Китай      3
40  LOC                 Германия      2
36  LOC        Красноярский край      2
33  LOC    Ленинградская область      2
65  LOC                 Петергоф      2
30  LOC    Нижегородская область      2
46  LOC                 Прометей      2
